# ResNet34 Model for Fashion MNIST
This model is the realization of the first object of this project, which was to create a deep neural network model which, when trained on the Fashion MNIST training dataset, could achieve an accuracy of 90% when classifying on the corresponding testing data.

This implementation is one called a residual network (ResNet). Most other networks 'linearily' pass down the information of an image through various levels of convulational filters and pooling layers. The uniquesnss of a ResNet is that it sums the output of convulational layers within 'blocks' of them as the features travel through. That way there is no issue of disappearing gradient, increasing the accuracy and performance of the model. 

Along with the arcitecture of the model, this notebook contains training and testing of the network, as applied to the Fashion MNIST data.

## Imports
Much of the resources required for the creation and evaluation of the model comes from Pytorch. Even the use of the FashionMNIST dataset is imported from Pytorch's version of it. 

For the visualisation of the results, the pandas and matplotlib's pyplot were used.

In [ ]:
from __future__ import print_function
import argparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torch.autograd import Variable
from typing import Tuple
from torch.utils.data import Dataset,DataLoader
# from sklearn.metrics import confusion_matrix, top_k_accuracy_score
import torchvision                                                       
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torchvision.transforms import ToTensor
from numpy import random as rd
import torch
import torch.nn as nn
import torch.optim as optim
import gc
from torch.utils.data.sampler import SubsetRandomSampler
import time

## Dataloader Block
Here, the Fashion MNIST dataset is loaded from torch's dataset library and split into its training and testing sets. The only transformation applied a ToTensor, for technical reasons. Any real image transformation would obscure information from an already low resolution, grayscale picture, with little performance benefit.

In [ ]:
# transform method
transform = transforms.Compose([
            transforms.ToTensor(),
    ])

# train data
trainData = datasets.FashionMNIST(root="./",
                                  train=True,
                                  transform=transform,
                                  download=True
                                  )
trainLoad = DataLoader(trainData, 
                       batch_size=512, 
                       shuffle=True, 
                       drop_last=False
                       )
# test data
testData = datasets.FashionMNIST(root="./",
                                  train=False,
                                  transform=transform,
                                  download=True
                                  )
testLoad = DataLoader(testData, 
                     batch_size=512, 
                     shuffle=True, 
                     drop_last=False
                     )

## Residual Block
The residual block holds the basic essence of the layers that compose the ResNet. The order by which each layer is initialized is the same order the layers are put in when forward is called and the model is built.

In [ ]:
class ResidualBlock(nn.Module):
    
    expansion = 4   # factor by which to expand the number of features per block
    
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        
        # First block's convolutional layer with a batch normalization and RELU activation
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

        # Second block's convolutional layer with batch normalization and RELU activation
        self.conv2 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels, kernel_size=7, stride=stride, padding=3),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )
        
        # Third block's convolutional layer with batch normalization
        self.conv3 = nn.Sequential(
            nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(out_channels*self.expansion)
        )

        #Finishing layers, with a downsample and activation
        self.downsample = downsample
        self.relu = nn.ReLU()
        self.out_channels=out_channels

    def forward(self, x):
        residual = x
        
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        if self.downsample != None:
            residual = self.downsample(residual)
            
        x += residual
        x = self.relu(x)
        return x

## Renset Block
This is the general model and the system by which the residual blocks are inserted. First is a input convolutional layer, followed by the layers of residual blocks, then a average pool to finish off with a fully connected linear layer to output the classification.

In [ ]:
# class for ResNet model that extend from nn.Module
class Resnet(nn.Module):
    
    # initialize the resnet model with inputted block type, list of blockNum 
    def __init__(self, block, blockList, input_num, output_num):
        super(Resnet, self).__init__()
        
        self.in_channels = 16  # Standard factor of feature channels to expand each block from
        
        # First convulotion layer with batch normalization, ReLU activation, and max pooling as this is the first layer from input
        self.conv1 = nn.Sequential(
            nn.Conv2d(input_num, 16, kernel_size=7, stride=2, padding=3),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        # Calling of the make layer functions to build middle of the model
        self.block0 = self._make_layer(block,  out_channels=16, blocksNum=blockList[0], stride=1)
        self.block1 = self._make_layer(block,  out_channels=32, blocksNum=blockList[1], stride=2)
        self.block2 = self._make_layer(block,  out_channels=64, blocksNum=blockList[2], stride=2)
        self.block3 = self._make_layer(block,  out_channels=128, blocksNum=blockList[3], stride=2)
        
        # apply 2D adaptive average pooling from 1 input to 1 plane
        # self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.avgpool = nn.AvgPool2d(1, 1)
        
        # flatten the data into 1 dimension
        self.flatten = nn.Flatten()
        
        # apply dropout to output with 60% percent chance
        self.drop = nn.Dropout(0.6)
        
        # connect 2048 input nodes into 10 output nodes
        self.fc = nn.Linear(128*4, output_num)

    # helper function that adds layer by layer along with the res block
    def _make_layer(self, block: ResidualBlock, out_channels, blocksNum, stride):
        downn_sample = None
        
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downn_sample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=3, stride=stride, padding=1),
                nn.BatchNorm2d(out_channels * block.expansion)
            )

        layers = []
        # layer that change the number of out channel, in will be inputed out
        layers.append(block(self.in_channels, out_channels, stride=stride, downsample=downn_sample))
        self.in_channels = out_channels * block.expansion
        
        # connected large output to smaller out 
        for _ in range(1, blocksNum):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)
    
    # forward function 
    def forward(self, x: ToTensor):
        x = self.conv1(x)
        
        x = self.block0(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        
        x = self.avgpool(x)
        x = self.drop(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

## Boiler Plate (Traing/Testing)
The method below is our primary means of training, testing, and evaluating the model. The arguments are the hyperparameters set for the use of the model. The model will run over the specified epochs with the batch size with optimization. Accuracy can be tested over the epochs from here.

In [ ]:
# number of epoch
epochNum=30
# learning rate
learningRate = .005
# weight decay
weightDecayRate = .001
# momentum
momentumAmount = .5
# setting up the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# setting up the model
# block using the ResidualBlock
# blockNums using the inputted list
# input_num of 1 for gray scaled, 3 for color
# output_num of 10 for 10 classes
model = Resnet(ResidualBlock, [3, 4, 6, 3], 1, 10).to(device)
# loss
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learningRate, weight_decay=weightDecayRate)
total_step = len(trainLoad)

#count
count = 0

epochs = [e for e in range(epochNum)]
losses = []
accuracies = []

predictions = []
label_list = []

for epoch in range(epochNum):
    for i, (images, labels) in enumerate(trainLoad):
        # move tensor to device
        images = images.to(device)
        labels = labels.to(device)
        
        # forward the output and calculate loss
        outputs = model(images)
        loss = criterion(outputs, labels)
        losses.append(loss.data)
        
        # backward the output and perform optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # deallocation
        del images, labels, outputs
        torch.cuda.empty_cache()
        gc.collect()
        
    print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochNum, loss.item()))
    
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in testLoad:
            images = images.to(device)
            labels = labels.to(device)
            label_list.append(labels)
            outputs = model(images)
            predicted = torch.max(outputs, 1)[1].to(device)
            predictions.append(predicted)
            total += labels.size(0)
            correct += (predicted == labels).sum()
            del images, labels, outputs
    accuracy = 100 * correct / total
    accuracies.append(accuracy)
    print('Accuracy of the network on the {} validation images: {} %'.format(10000, accuracy))

## Results Graphs
Below are some visualizations the classification results gained from the use of the ResNet upon FashionMNIST.

Loss Graph

In [ ]:
plt.plot(epochs, losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Epochs vs Loss")
plt.show()

Accuracy Graph

In [ ]:
plt.plot(epochs, accuracies)
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Epochs vs Accuracy")
plt.show()

Confusion Matrix

In [ ]:
from itertools import chain 

predictions_list = [predictions[i].tolist() for i in range(len(predictions))]
labels_list = [label_list[i].tolist() for i in range(len(label_list))]
predictions_list = list(chain.from_iterable(predictions))
labels_list = list(chain.from_iterable(labels_list))